In [43]:
import os
import pandas as pd

#Here we are importing the Amazon Data set

# ✅ Set working directory to repo root
os.chdir("/Users/arvidsson/Desktop/Predicting Customer Dissatisfaction/Safeify/summer-2025-safeify")

# ✅ Define correct file path relative to repo root
file_path = "Data/amazon_meta.json"

# ✅ Confirm and load the data
print("🛠️ Now working from:", os.getcwd())
print("✅ Exists?", os.path.exists(file_path))

print("📥 Loading Amazon metadata...")
amazon_df = pd.read_json(file_path, lines=True)
print("✅ Loaded", len(amazon_df), "rows")
print("📦 Columns:", amazon_df.columns.tolist())
print("🧪 Sample:")
print(amazon_df.head(3))


🛠️ Now working from: /Users/arvidsson/Desktop/Predicting Customer Dissatisfaction/Safeify/summer-2025-safeify
✅ Exists? True
📥 Loading Amazon metadata...
✅ Loaded 633883 rows
📦 Columns: ['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2', 'brand', 'feature', 'rank', 'also_view', 'main_cat', 'similar_item', 'date', 'price', 'asin', 'imageURL', 'imageURLHighRes', 'details']
🧪 Sample:
                                  category tech1  \
0  [Toys & Games, Puzzles, Jigsaw Puzzles]         
1                                       []         
2                                       []         

                                         description fit  \
0  [Three Dr. Suess' Puzzles: Green Eggs and Ham,...       
1  [<b>Prepare to be Afraid!</b><br /><br />The B...       
2                                                 []       

                                            title also_buy tech2  \
0  Dr. Suess 19163 Dr. Seuss Puzzle 3 Pack Bundle       []         
1     Pa

In [44]:
# Importing all Recall Data, combining the three files

def load_clean_csv(path):
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # ✅ Find the first line that contains all expected headers (starts with 'Report No.')
    header_index = next(i for i, line in enumerate(lines) if 'Report No.' in line)

    # ✅ Load CSV from that line forward
    return pd.read_csv(path, skiprows=header_index)

# ✅ Paths to all three files
recall_files = [
    "Data/Current Version of Toys Incidence+Recall/Toysandchildren_ArtsandCrafts.csv",
    "Data/Current Version of Toys Incidence+Recall/Toysandchildren_Riding_Toys.csv",
    "Data/Current Version of Toys Incidence+Recall/Toysandchildren_Toys.csv"
]

# ✅ Load all cleanly
recall_dfs = [load_clean_csv(path) for path in recall_files]

# ✅ Merge into single df
recalls_df = pd.concat(recall_dfs, ignore_index=True)

# ✅ Preview
print("✅ Combined recall records:", len(recalls_df))
print("📦 Columns:", recalls_df.columns.tolist())
recalls_df.head(3)


✅ Combined recall records: 2514
📦 Columns: ['Report No.', 'Report Date', 'Sent to Manufacturer / Importer / Private Labeler', 'Publication Date', 'Category of Submitter', 'Product Description', 'Product Category', 'Product Sub Category', 'Product Type', 'Product Code', 'Manufacturer / Importer / Private Labeler Name', 'Brand', 'Model Name or Number', 'Serial Number', 'UPC', 'Date Manufactured', 'Manufacturer Date Code', 'Retailer', 'Retailer State', 'Purchase Date', 'Purchase Date Is Estimate', 'Incident Description', 'City', 'State', 'ZIP', 'Location', '(Primary) Victim Severity', "(Primary) Victim's Sex", 'My Relation To The (Primary) Victim', "(Primary) Victim's Age (years)", 'Submitter Has Product', 'Product Was Damaged Before Incident', 'Damage Description', 'Damage Repaired', 'Product Was Modified Before Incident', 'Have You Contacted The Manufacturer', 'If Not Do You Plan To', 'Answer Explanation', 'Company Comments', 'Associated Report Numbers']


,Report No.,Report Date,Sent to Manufacturer / Importer / Private Labeler,Publication Date,Category of Submitter,Product Description,Product Category,Product Sub Category,Product Type,Product Code,...,Submitter Has Product,Product Was Damaged Before Incident,Damage Description,Damage Repaired,Product Was Modified Before Incident,Have You Contacted The Manufacturer,If Not Do You Plan To,Answer Explanation,Company Comments,Associated Report Numbers
0,20231002-D13C7-2147344911,10/2/2023,10/27/2023,11/20/2023,Consumer,Slime globe with colored spheres which resembl...,Toys & Children,Arts & Crafts,Molding Compounds (1376),1376,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TOYSMITH: Thank you for the opportunity to rev...,NaN
1,20180126-BBF18-2147393362,1/26/2018,4/13/2018,4/27/2018,Consumer,Slime kit from Nickelodeon by Cra-Z-Art,Toys & Children,Arts & Crafts,Molding Compounds (1376),1376,...,Yes,No,NaN,NaN,No,Yes,NaN,I already have explained.,We at Cra-Z-Art® are very sorry that the cons...,NaN
2,20141231-7C153-2147437145,12/31/2014,1/9/2015,1/26/2015,Consumer,Lalaloopsy Color Me Doll ( Squiggles N. Shapes...,Toys & Children,Arts & Crafts,Crayons or Chalk (5010),5010,...,NaN,NaN,NaN,NaN,NaN,No,No,NaN,The chalk markers included with this item are ...,NaN


In [45]:
# Features we can use for fuzzy matching.

print(amazon_df.columns)
print(recalls_df.columns.tolist())

Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2',
       'brand', 'feature', 'rank', 'also_view', 'main_cat', 'similar_item',
       'date', 'price', 'asin', 'imageURL', 'imageURLHighRes', 'details'],
      dtype='object')
['Report No.', 'Report Date', 'Sent to Manufacturer / Importer / Private Labeler', 'Publication Date', 'Category of Submitter', 'Product Description', 'Product Category', 'Product Sub Category', 'Product Type', 'Product Code', 'Manufacturer / Importer / Private Labeler Name', 'Brand', 'Model Name or Number', 'Serial Number', 'UPC', 'Date Manufactured', 'Manufacturer Date Code', 'Retailer', 'Retailer State', 'Purchase Date', 'Purchase Date Is Estimate', 'Incident Description', 'City', 'State', 'ZIP', 'Location', '(Primary) Victim Severity', "(Primary) Victim's Sex", 'My Relation To The (Primary) Victim', "(Primary) Victim's Age (years)", 'Submitter Has Product', 'Product Was Damaged Before Incident', 'Damage Description', 'Damage Repaired

The code below creates a dictionary with each entry being a tuple containing two dataframes 
(recalls_df['Brand'], matched_amazon_by_brand) 
Here matched_amazon_by_brand is a Dataframe containing all Amazon entries that Fuzzy match 'brand' with 'Brand' above a certain threshold, here it is 90. To save time, and if you like to run this as an experiment, the block below uses a sample of 500 brands and 30 000 Amazon entries. The full dataset is explored at the end of this notebook.

In [46]:
from rapidfuzz import fuzz
import pandas as pd

# Step 1: Clean and sample brand names from recalls_df
sampled_brands = pd.Series(recalls_df['Brand'].dropna().unique()[:500])
sampled_brands = sampled_brands.astype(str).str.lower().str.strip()

# Step 2: Sample and clean Amazon data
amazon_sample = amazon_df.sample(30000, random_state=42).copy()
amazon_sample['brand'] = amazon_sample['brand'].astype(str).str.lower().str.strip()

# Step 3: Clean 'Brand' in recalls_df for matching
recalls_df['Brand'] = recalls_df['Brand'].astype(str).str.lower().str.strip()

# Step 4: Prepare dictionary to store tuples (Amazon matches, recall row(s))
matched_amazon_by_brand = {}

# Step 5: For each brand, store Amazon matches + associated recall row(s)
for recall_brand in sampled_brands:
    # Find Amazon entries that match this brand (fuzzy match ≥ 90)
    matched_rows = amazon_sample[amazon_sample['brand'].apply(
        lambda x: fuzz.token_sort_ratio(x, recall_brand) >= 90
    )]

    # Get corresponding recall row(s) for this brand
    recall_rows = recalls_df[recalls_df['Brand'] == recall_brand]

    # Only store if there are matched Amazon entries
    if not matched_rows.empty:
        matched_amazon_by_brand[recall_brand] = (recall_rows.copy(), matched_rows.copy())

# ✅ matched_amazon_by_brand now maps each brand → (recall_rows_df, matched_amazon_rows_df)

# Optional: Print summary
print(f"✅ Found matches for {len(matched_amazon_by_brand)} recall brand entries.")


✅ Found matches for 130 recall brand entries.


In [47]:
# 🔢 Stats on number of Amazon listings matched per recall brand
match_counts = [len(amazon_df) for _, amazon_df in matched_amazon_by_brand.values()]

if match_counts:
    avg_matches = sum(match_counts) / len(match_counts)
    min_matches = min(match_counts)
    max_matches = max(match_counts)

    print(f"📊 Average # of Amazon listings matched per brand: {avg_matches:.2f}")
    print(f"🔽 Minimum # of matches: {min_matches}")
    print(f"🔼 Maximum # of matches: {max_matches}")
else:
    print("⚠️ No matches found to compute stats.")


📊 Average # of Amazon listings matched per brand: 29.73
🔽 Minimum # of matches: 1
🔼 Maximum # of matches: 529


Now the idea is to do Fuzzy matching on each entry of the Dictionary we have created, using 'Product Description' from the recall data and some interesting category from the Amazon data. Let's have a look on 'title', 'feature' and 'description'.

In [48]:
i = 10 #choosing brand nr 10 from dictionary, change i to see different brands
brand = sampled_brands.iloc[i]
print(f"\n🔍 Brand selected (index {i}): '{brand}'")

if brand in matched_amazon_by_brand:
    recall_df, amazon_df_matches = matched_amazon_by_brand[brand]

    # Display the dataframes (normal truncated view)
    display(recall_df)
    display(amazon_df_matches) 

    # Display full text of selected columns
    print("\n📌 Product Descriptions (Recall):")
    print(recall_df['Product Description'].to_string(index=False))

    print("\n📌 Amazon 'description':")
    print(amazon_df_matches['description'].to_string(index=False))

    print("\n📌 Amazon 'title':")
    print(amazon_df_matches['title'].to_string(index=False))

    print("\n📌 Amazon 'feature':")
    print(amazon_df_matches['feature'].to_string(index=False))
else:
    print(f"No matches found for brand: {brand}")




🔍 Brand selected (index 10): 'creative kids'


,Report No.,Report Date,Sent to Manufacturer / Importer / Private Labeler,Publication Date,Category of Submitter,Product Description,Product Category,Product Sub Category,Product Type,Product Code,...,Submitter Has Product,Product Was Damaged Before Incident,Damage Description,Damage Repaired,Product Was Modified Before Incident,Have You Contacted The Manufacturer,If Not Do You Plan To,Answer Explanation,Company Comments,Associated Report Numbers
12,20150902-1BB82-2147428770,9/2/2015,9/11/2015,9/25/2015,Consumer,Create Your Own Magic Bouncy Balls: Create fu...,Toys & Children,Arts & Crafts,Metal or Plastic Molding Sets (1319),1319,...,No,No,NaN,NaN,No,No,No,NaN,NaN,NaN


,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
517003,"[Toys & Games, Arts & Crafts, Craft Kits, Sand...",,[Great gift idea too!],,X-Sand Kit,[],,creative kids,"[Make Cool Sand Creations!, Includes 4 X-Sand ...","[>#1,272,427 in Toys & Games (See Top 100 in T...",[],Toys & Games,,,$6.32,B00YCW8ZDG,[],[],{}
380483,"[Toys & Games, Arts & Crafts, Craft Kits, Sand...",,[<b>Lifes a beach with this sensational sand a...,,Creative Kids SSANDART (Glitter Sand Art),[],,creative kids,[CREATE STUNNING SAND ART All-in-One Super Sa...,"[>#217,704 in Toys & Games (See Top 100 in Toy...",[],Toys & Games,"class=""a-bordered a-horizontal-stripes a-spa...",,$19.96,B00H9CUKMU,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}



📌 Product Descriptions (Recall):
Create Your Own Magic Bouncy Balls:  Create fun...

📌 Amazon 'description':
                            [Great gift idea too!]
[<b>Lifes a beach with this sensational sand ar...

📌 Amazon 'title':
                               X-Sand Kit
Creative Kids SSANDART (Glitter Sand Art)

📌 Amazon 'feature':
[Make Cool Sand Creations!, Includes 4 X-Sand T...
[CREATE STUNNING SAND ART  All-in-One Super San...


After some inspection and experimentation it seems like the 'title' category is the best option. Below we use a fuzzy match of 80, which on the brand -- matched dataframes seem to do the right thing, look at the code below. I originally used 70, which might have been to low. 

In [49]:
from rapidfuzz import fuzz
import pandas as pd

# Loop over all brands in sampled_brands
for i, brand in enumerate(sampled_brands):
    if brand in matched_amazon_by_brand:
        recall_df, amazon_df_matches = matched_amazon_by_brand[brand]

        # Clean and prepare text columns
        recall_df['Product Description'] = recall_df['Product Description'].astype(str).fillna('').str.lower().str.strip()
        amazon_df_matches['title'] = amazon_df_matches['title'].astype(str).fillna('').str.lower().str.strip()

        matches = []

        # Compare each product description in recall_df to each Amazon title
        for recall_idx, recall_desc in recall_df['Product Description'].items():
            for amazon_idx, amazon_title in amazon_df_matches['title'].items():
                score = fuzz.token_set_ratio(recall_desc, amazon_title)
                if score >= 80:
                    matches.append({
                        'recall_idx': recall_idx,
                        'amazon_idx': amazon_idx,
                        'recall_desc': recall_desc,
                        'amazon_title': amazon_title,
                        'score': score
                    })

        if matches:
            matched_desc_df = pd.DataFrame(matches).sort_values(by='score', ascending=False)
            print(f"\n🔍 [{i}] Brand: '{brand}'")
            print(f"✅ Found {len(matched_desc_df)} title matches")
            display(matched_desc_df.head(2))  # Limit display to top 2
# You will find a list of all matches showing the title that was matched to the product description. 
# You can scroll through and see if it is reasonable to you           



🔍 [4] Brand: 'nickelodeon'
✅ Found 2 title matches


,recall_idx,amazon_idx,recall_desc,amazon_title,score
0,1914,334226,childs bed tent with push light,dora the explorer bed tent with push light fea...,87.272727
1,2142,270893,"teenage mutant ninja turtles teepee, pillow & ...",teenage mutant ninja turtles foot soldier,81.159420



🔍 [7] Brand: 'crayola'
✅ Found 7 title matches


,recall_idx,amazon_idx,recall_desc,amazon_title,score
0,8,486730,crayon,"crayola tin crayon box, holds 64 crayons, 6&qu...",100.0
1,8,431986,crayon,crayola &quot;metallic magic&quot; 16 count cr...,100.0



🔍 [12] Brand: 'playskool'
✅ Found 3 title matches


...

,recall_idx,amazon_idx,recall_desc,amazon_title,score
1,642,155219,vtech yellow count and learn school busbrand v...,vtech - count and learn school bus,90.322581
0,583,359617,vtech helicopter toy,french language vtech explore helicopter,88.888889



🔍 [473] Brand: 'baby einstein'
✅ Found 3 title matches


,recall_idx,amazon_idx,recall_desc,amazon_title,score
0,1225,521112,this product is called baby einstein take alon...,baby einstein carry along caterpillar toy,92.105263
1,1225,127243,this product is called baby einstein take alon...,baby einstein music mirror,84.444444



🔍 [494] Brand: 'sesame street'
✅ Found 2 title matches


,recall_idx,amazon_idx,recall_desc,amazon_title,score
0,439,229306,elmo with a guitar,sesame street let's rock elmo with bonus guitar,94.117647
1,668,98079,red elmo plush toy,sesame street plush elmo doll in overalls - toy,87.500000


This looks pretty good! Lets see how many matches we have. 

In [50]:

#Loops over all (recall_df, amazon_df_matches) pairs from matched_amazon_by_brand

#Concatenates all Amazon matches into one DataFrame

#Drops duplicates using asin, assuming asin uniquely identifies a listing

#Returns the count

# Collect all matched Amazon listings across all brands
all_amazon_matches = pd.concat(
    [amazon_df_matches for _, amazon_df_matches in matched_amazon_by_brand.values()],
    ignore_index=True
)

# Drop duplicates by ASIN to count unique listings
total_unique_amazon_matches = all_amazon_matches.drop_duplicates(subset='asin').shape[0]

print(f"📦 Total unique asin nr Amazon listings matched by brand: {total_unique_amazon_matches}")


📦 Total unique asin nr Amazon listings matched by brand: 3456


We see that on a sample of 30 000 listings a full 3456 was matched to recall data, and this was when only 1/4 of the brands appearing in the recall data where used. 

I will now run this code on the full Amazon data and using the full recall data, this took me 8 min 45s. 

In [51]:
from rapidfuzz import fuzz
import pandas as pd

# Ensure brand fields are cleaned
amazon_df['brand'] = amazon_df['brand'].astype(str).str.lower().str.strip()
recalls_df['Brand'] = recalls_df['Brand'].astype(str).str.lower().str.strip()

# 🧼 Create the list of all unique brands from the entire recalls_df
all_recall_brands = pd.Series(recalls_df['Brand'].dropna().unique())

# Prepare dictionary to store (recall_rows, amazon_matches)
matched_amazon_by_brand = {}

# Loop over all brands from the full recall list
for recall_brand in all_recall_brands:
    # Find Amazon entries where the brand fuzzy-matches (≥ 90)
    matched_rows = amazon_df[amazon_df['brand'].apply(
        lambda x: fuzz.token_sort_ratio(x, recall_brand) >= 90
    )]

    # Get recall rows with that brand
    recall_rows = recalls_df[recalls_df['Brand'] == recall_brand]

    if not matched_rows.empty:
        matched_amazon_by_brand[recall_brand] = (recall_rows.copy(), matched_rows.copy())

# ✅ Summary
print(f"✅ Found matches for {len(matched_amazon_by_brand)} recall brand entries.")


✅ Found matches for 586 recall brand entries.


When matching with 70, I got around 6000 matches, below we do for 80

In [52]:
# Step 2: Tag Amazon entries that match product descriptions

# Initialize column to 0
amazon_df['Is match'] = 0

# Store indices of matched rows
matched_amazon_indices = set()

# Loop over matched brands
for brand, (recall_df, amazon_df_matches) in matched_amazon_by_brand.items():
    # Clean relevant columns
    recall_df['Product Description'] = recall_df['Product Description'].astype(str).fillna('').str.lower().str.strip()
    amazon_df_matches['title'] = amazon_df_matches['title'].astype(str).fillna('').str.lower().str.strip()

    for recall_desc in recall_df['Product Description']:
        for idx, title in amazon_df_matches['title'].items():
            score = fuzz.token_set_ratio(recall_desc, title)
            if score >= 80:
                matched_amazon_indices.add(idx)

# Update original amazon_df with match indicator
amazon_df.loc[amazon_df.index.isin(matched_amazon_indices), 'Is match'] = 1

# ✅ Summary
print(f"✅ Total matched Amazon listings: {len(matched_amazon_indices)}")
print("🧾 Is match counts:\n", amazon_df['Is match'].value_counts())


✅ Total matched Amazon listings: 2153
🧾 Is match counts:
 Is match
0    631730
1      2153
Name: count, dtype: int64


In [53]:
print(f"📦 Total products in the dataset: {len(amazon_df)}")

📦 Total products in the dataset: 633883


In [55]:
# Collect examples to show
example_matches = []

for brand, (recall_df, amazon_df_matches) in matched_amazon_by_brand.items():
    recall_df['Product Description'] = recall_df['Product Description'].astype(str).fillna('').str.lower().str.strip()
    amazon_df_matches['title'] = amazon_df_matches['title'].astype(str).fillna('').str.lower().str.strip()

    for recall_desc in recall_df['Product Description']:
        for idx, title in amazon_df_matches['title'].items():
            if idx in matched_amazon_indices:  # Only show already matched indices
                score = fuzz.token_set_ratio(recall_desc, title)
                if score >= 80:
                    example_matches.append({
                        'brand': brand,
                        'amazon_index': idx,
                        'amazon_title': title,
                        'recall_description': recall_desc,
                        'score': score
                    })
                if len(example_matches) >= 10:
                    break
        if len(example_matches) >= 10:
            break
    if len(example_matches) >= 10:
        break

# Convert to DataFrame for display
examples_df = pd.DataFrame(example_matches)
print("\n📦 First 10 example matches:")
print(examples_df[['brand', 'amazon_title', 'recall_description', 'score']])



📦 First 10 example matches:
         brand                                       amazon_title  \
0     play doh  play doh disney princess design a dress boutiq...   
1  nickelodeon  1 x turtles teenage mutant ninja turtles mutag...   
2  nickelodeon  nickelodeon teenage mutant ninja turtle heli b...   
3  nickelodeon  dora the explorer bed tent with push light fea...   
4  nickelodeon              teenage mutant ninja turtles splinter   
5  nickelodeon              teenage mutant ninja turtles leonardo   
6  nickelodeon              teenage mutant ninja turtles shredder   
7  nickelodeon          teenage mutant ninja turtles foot soldier   
8  nickelodeon            teenage mutant ninja turtles cups 4 set   
9  nickelodeon             teenage mutant ninja turtles kickboard   

                                  recall_description       score  
0                                           play doh  100.000000  
1  playmate's teenage mutant ninja turtle nickelo...   84.033613  
2  playmat

Match is definetely not perfect but maybe good enough?